In [71]:
from utils import normalize_df, create_train_test_val_df
import os
notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
data_dir = os.path.join(root_dir, 'data')
import numpy as np
import pandas as pd
from IPython.display import display_html
from copy import deepcopy
import pickle
# from utils.utils

# Model 1.0
This notebook will act as an interactive tutorial for our Live Win Probability Model. This "model" is actual comprised of 3 separate models that "stack" on each other.
1. Play and drive outcome models
    * technically this is two separate models:
        * Play outcome (first down, field goal made, field goal missed, touchdown, turnover, and none/other)
            * only using the first down prediction from the output of this model
        * Drive outcome (Clock, field goal made, field goal missed, punt, safety, touch down, turnover, turnover on downs)
    * outputs for both models will be a series of probabilities for each class that all add up to 1
2. End of regulation score differential model
    * Dealing with overtime later, we want to predict how the score differential will change by the end of regulation.
        * i.e., if the current score differential (home score - away score) is -3 and the end of regulation score differential is -10, the target value will be -7
    * Output of this will be a series of probabilities from for all score differential possibilities from -35 to 35 (outputs <-35 or >35 will be set to -35/35 respectively)
3. End of regulation score total model
    * Similar concept to the score differential model
    * Again, we're using the change in end of regulation score total as the target value
    * Outputs will be a series of probabilites for classes from 0 to 83 (outputs will be capped at 83)

## Data
Let's take a look at the data that we are pulling from oracle
* First we have event_df and odds_df
* event_df is the play by play data mixed with some import game information
* Odds data has vegas predictions for almost all the games in the set (missing games will be given the average vegas spread and over/under)
    * The spread and over/under are merged with the event table to give us our pre-game priors
    * some games have multiple odds so duplicates are removed


In [10]:
event_df = pd.read_parquet(os.path.join(data_dir, "event_data.parquet"))
event_df = event_df.drop_duplicates(["nevent", "game_code"]).reset_index(drop=True)
odds_df = pd.read_parquet(os.path.join(data_dir, "odds_data.parquet"))
odds_df = odds_df.drop_duplicates("game_code")
event_df[["cur_spread", "cur_over_under"]] = event_df.merge(odds_df, how="left", on="game_code")[["cur_spread", "cur_over_under"]].fillna({"cur_spread": np.mean(odds_df["cur_spread"]), "cur_over_under": np.mean(odds_df["cur_over_under"])})
pd.set_option("display.max_columns", None)
display_html(event_df)

,game_code,game_date,season,home_team_id,home_team,home_team_abbrev,away_team_id,away_team,away_team_abbrev,home_final_score,away_final_score,final_score_diff,end_of_regulation_score_diff,home_rest_of_game_score,away_rest_of_game_score,end_of_regulation_score_diff_change,home_score_added,away_score_added,current_score_diff,current_score_total,home_start_score,away_start_score,home_team_outcome,home_team_win,draw,away_team_win,nevent,quarter,overtime,home_team_has_ball,off_team_id,def_team_id,kick_off,punt,point_after_kick,two_point_attempt,field_goal_attempt,off_start_score,off_end_score,off_score_change,def_start_score,def_end_score,def_score_change,play_counts,efficiency_counts,from_scrimmage,first_down,scoring_play,possession_change,continuation,event_name,event_id,yards_gained,drive_outcome_id,drive_outcome_desc,down,ytg,yd_from_goal,drive_id,drive_start,play_start_time,cur_spread,cur_over_under
0,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,1,1,0,0,363,351,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,Kick Off,5,73.0,NaN,None,0,-1,70,NaN,3600,900.0,-4.5,41.5
1,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,2,1,0,1,351,363,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,Kick Off Return,6,19.0,NaN,None,0,-1,103,NaN,3600,900.0,-4.5,41.5
2,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,3,1,0,1,351,363,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,Run,4,3.0,37.0,TD,1,10,84,1.0,3600,895.0,-4.5,41.5
3,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,4,1,0,1,351,363,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,Incomplete Pass,2,0.0,37.0,TD,2,7,81,1.0,3600,860.0,-4.5,41.5
4,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,5,1,0,1,351,363,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,Pass Completion,1,8.0,37.0,TD,3,7,81,1.0,3600,854.0,-4.5,41.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816793,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,201,4,0,1,327,343,0,0,0,0,0,20,20,0,23,23,0,1,1,1,0,0,0,0,Run,4,0.0,40.0,Downs,3,1,49,13.0,85,48.0,4.5,48.5
816794,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,202,4,0,1,327,343,0,0,0,0,0,20,20,0,23,23,0,1,0,0,0,0,0,0,Offense Timeout,57,NaN,40.0,Downs,4,1,49,13.0,85,43.0,4.5,48.5
816795,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,203,4,0,1,327,343,0,0,0,0,0,20,20,0,23,23,0,1,1,1,0,0,1,0,Incomplete Pass,2,0.0,40.0,Downs,4,1,49,13.0,85,43.0,4.5,48.5
816796,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,204,4,0,0,343,327,0,0,0,0,0,23,23,0,20,20,0,1,1,1,0,0,0,0,Run,4,-1.0,39.0,End Game,1,10,51,13.0,39,39.0,4.5,48.5


Adding timeouts remaining for both teams and time left in game

In [34]:
event_df["half"] = round((event_df["quarter"] + 0.01) / 2)
event_df["home_timeout"] = np.where(((event_df["event_id"]==57)&(event_df["home_team_has_ball"]==1))|((event_df["event_id"]==58)&(event_df["home_team_has_ball"]==0)), 1, 0)
event_df["away_timeout"] = np.where(((event_df["event_id"]==57)&(event_df["home_team_has_ball"]==0))|((event_df["event_id"]==58)&(event_df["home_team_has_ball"]==1)), 1, 0)
event_df["home_timeouts_remaining"] = np.clip(3 - event_df.groupby(["game_code", "half"])["home_timeout"].cumsum(), 0, 3)
event_df["away_timeouts_remaining"] = np.clip(3 - event_df.groupby(["game_code", "half"])["away_timeout"].cumsum(), 0, 3)
event_df["time_left_in_game"] = np.where(event_df["quarter"] <= 4, event_df["play_start_time"] + (4 - event_df["quarter"]) * 900, event_df["play_start_time"])
# event_df["time_elapsed"] = 900 - event_df["play_start_time"] + (event_df["quarter"] - 1) * 900


* Our PBP will have multiple rows for one play, so if there's a fumble then recovery by offense and a touchdown, 
* that could have 2-3 rows of data and the touchdown wouldn't show up as being apart of the original play 
    * plays would look like this: 1. Run, 2. Fumble, 3. Offense Recovers the ball (TD)
* So what we've done here is ensure that plays that are "continuation" that end in a touchdown, give a TD=True for all of the plays in the sequence
* After that is taken care of we can setup all of the labels for play and drive description

In [11]:
event_df["sequence"] = event_df["continuation"].groupby(event_df["continuation"].eq(0).cumsum()).cumsum()
event_df["play_start_id"] = event_df["nevent"] - event_df["sequence"]

turnover_ids = [9, 16]
event_df["turnover"] = np.where(event_df["event_id"].isin(turnover_ids), 1, 0)
event_df["touchdown_scored"] = np.where(event_df["home_score_added"]+event_df["away_score_added"]>=6, 1, 0)
event_df["fieldgoal_made"] = np.where(event_df["home_score_added"]+event_df["away_score_added"]==3, 1, 0)

play_outcome_aggregate =event_df[["game_code", "play_start_id", "turnover", "touchdown_scored", "fieldgoal_made", "first_down"]].groupby(["game_code", "play_start_id"], as_index=False).sum()
event_df["touchdown_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["touchdown_scored_y"], 0, 1)
event_df["turnover_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["turnover_y"], 0, 1)
event_df["field_goal_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["fieldgoal_made_y"], 0, 1)
event_df["first_down_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["first_down_y"], 0, 1)



event_df["play_outcome"] = (
    np.where((event_df["turnover_in_play"]==1), "turnover",
    # (event_df["touchdown_in_play"]==1)&(event_df["turnover_in_play"]==1), "defensive_touchdown", 
    np.where((event_df["field_goal_in_play"]==1), "field_goal_made",
    np.where((event_df["field_goal_attempt"]==1)&(event_df["field_goal_in_play"]==0), "field_goal_missed",
    np.where((event_df["first_down_in_play"]==1)&(event_df["touchdown_in_play"]==0)&(event_df["turnover_in_play"]==0), "first_down",
    np.where((event_df["touchdown_in_play"]==1)&(event_df["turnover_in_play"]==0), "offensive_touchdown", "none")))))
)
drive_description_matrix = {
    7: "punt",
    9: "turnover",
    14: "turnover",
    17: "field_goal_made",
    18: "punt",
    20: "safety",
    35: "field_goal_missed",
    36: "field_goal_missed",
    37: "touch_down",
    38: "clock",
    39: "clock",
    40: "turnover_on_downs",
    42: "field_goal_made",
    51: "clock",
}
event_df["drive_outcome_desc_basic"] = event_df["drive_outcome_id"].map(drive_description_matrix)

# event_df["drive_outcome"] = np.where(
#     (event_df["touchdown_in_drive"]==1)&(event_df["turnover_in_drive"]==1), "defensive_touchdown", 
#     np.where((event_df["touchdown_in_drive"]==1)&(event_df["turnover_in_drive"]==0), "offensive_touchdown",
#     np.where((event_df["field_goal_in_drive"]==1), "field_goal_made",
#     np.where((event_df["touchdown_in_drive"]==0)&(event_df["turnover_in_drive"]==1), "turnover", "none"
# ))))
game_end_of_regulation_total_score = event_df[event_df.overtime==0].groupby("game_code", as_index=False).max()[["game_code", "home_start_score", "away_start_score"]]
game_end_of_regulation_total_score["end_of_regulation_score_total"] = game_end_of_regulation_total_score["home_start_score"] + game_end_of_regulation_total_score["away_start_score"]
# event_df["end_of_regulation_score_total_diff"] = 
event_df["end_of_regulation_score_total_diff"] = (
    event_df.merge(game_end_of_regulation_total_score, on="game_code")["end_of_regulation_score_total"]
    - (event_df["home_start_score"] + event_df["away_start_score"])
)

/var/folders/dq/tcxv3s4j7y585_4jhfbdvrx00000gn/T/ipykernel_3313/4279417664.py:50: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  game_end_of_regulation_total_score = event_df[event_df.overtime==0].groupby("game_code", as_index=False).max()[["game_code", "home_start_score", "away_start_score"]]


### Data Manipulation
* We need to do a little bit of data manipulation to get the values we need, but we don't want to "overwrite" the values in event_df so we'll make copy of it called model_df
* time left in half is added
* from_scrimmage is changed so that PATs and two point conversions are not included
* down, ytg, and yd_from_goal are changed so that all non-scrimmage plays are changed to a default "null" value
* home_team_has_ball is change so that when kickoffs occur, the team receiving is the one that is in possession of the ball

### Data Subset
* Removing continuation plays that we mentioned before, so that each snap has just one target
* Remove plays where the down is equal to 0 
* Remove plays from scrimmage that did not count (e.g., plays that were waved off by penalties)
* scrimmage_plays_we_want is event_id of all the scrimmage plays that *aren't* timeouts, end of quarters, and the two minute warning.
* Remove all NA values for the feature inputs and target
* Remove all plays that are not from scrimmage
* Remove all overtime plays

In [69]:
model_df = deepcopy(event_df)
model_df["time_left_in_half"] = event_df["time_left_in_game"] - ((2 - event_df["half"]) * 1800)
model_df["from_scrimmage"] = np.where(event_df["event_id"].isin([22, 47, 52, 53, 54, 55, 56]), 0, event_df["from_scrimmage"])
model_df["down"] = np.where(model_df["from_scrimmage"] == 0, 0, event_df["down"])
model_df["ytg"] = np.where(model_df["from_scrimmage"] == 0, -1, event_df["ytg"])
model_df["yd_from_goal"] = np.where(model_df["from_scrimmage"] == 0, -1, event_df["yd_from_goal"])
model_df["home_team_has_ball"] = np.where(event_df["event_id"].isin([5]), 1 - event_df["home_team_has_ball"], event_df["home_team_has_ball"])
scrimmage_plays_we_want = [1, 2, 3, 4, 7, 9, 14, 17, 18, 35]

input_names = [
    'time_left_in_half',
    'half',
    'current_score_diff',
    'current_score_total',
    'cur_spread',
    'cur_over_under',
    'home_timeouts_remaining',
    'away_timeouts_remaining',
    'punt',
    'field_goal_attempt',
    'ytg',
    'yd_from_goal',
    'down',
    'home_team_has_ball',
]
mask_model = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df.play_counts==1)&
    (model_df.event_id.isin(scrimmage_plays_we_want))&
    (model_df[input_names].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)


Let's take a look at what the input features and output features look like. 

Event Name and yards gained is included to help interpret what is going on. 

This is the first 2 drives of the first game in the dataset

In [70]:
model_df[mask_model][input_names + ["event_name", "yards_gained", "play_outcome", "drive_outcome_desc_basic"]].head(15)

,time_left_in_half,half,current_score_diff,current_score_total,cur_spread,cur_over_under,home_timeouts_remaining,away_timeouts_remaining,punt,field_goal_attempt,ytg,yd_from_goal,down,home_team_has_ball,event_name,yards_gained,play_outcome,drive_outcome_desc_basic
2,1795.0,1.0,0,0,-4.5,41.5,3,3,0,0,10,84,1,1,Run,3.0,none,touch_down
3,1760.0,1.0,0,0,-4.5,41.5,3,3,0,0,7,81,2,1,Incomplete Pass,0.0,none,touch_down
4,1754.0,1.0,0,0,-4.5,41.5,3,3,0,0,7,81,3,1,Pass Completion,8.0,first_down,touch_down
5,1723.0,1.0,0,0,-4.5,41.5,3,3,0,0,10,73,1,1,Run,3.0,none,touch_down
6,1676.0,1.0,0,0,-4.5,41.5,3,3,0,0,7,70,2,1,Pass Completion,30.0,first_down,touch_down
7,1650.0,1.0,0,0,-4.5,41.5,3,3,0,0,10,40,1,1,Pass Completion,19.0,first_down,touch_down
8,1612.0,1.0,0,0,-4.5,41.5,3,3,0,0,10,21,1,1,Run,4.0,none,touch_down
9,1562.0,1.0,0,0,-4.5,41.5,3,3,0,0,6,17,2,1,Incomplete Pass,0.0,none,touch_down
10,1555.0,1.0,0,0,-4.5,41.5,3,3,0,0,6,17,3,1,Pass Completion,11.0,first_down,touch_down
11,1517.0,1.0,0,0,-4.5,41.5,3,3,0,0,6,6,1,1,Incomplete Pass,0.0,none,touch_down


### Loading in the Models and Creating Play/Drive Predictions
* For this exercise we won't be training the models, just loading saved models and then using them to make predictions
* We're now including plays that didn't wind up counting
* In addition, each prediction will be split up between home and away. So if the home team has the ball the predictions for the away team play/drive outcomes are going to be set to 0


Let's take a look at how the home team predictions look for our dataset

In [107]:
search_rf_play_outcome = pickle.load(open(os.path.join(root_dir, "models/search_rf_play_outcome.p"), 'rb'))
search_rf_drive_outcome = pickle.load(open(os.path.join(root_dir, "models/search_rf_drive_outcome.p"), 'rb'))
search_rf_play_outcome.best_estimator_.verbose = 0
search_rf_drive_outcome.best_estimator_.verbose = 0


mask_model_predict = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df[input_names].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)


search_rf_play_class_names = ["search_rf_play_" + x for x in search_rf_play_outcome.classes_]
search_rf_drive_class_names = ["search_rf_drive_" + x for x in search_rf_drive_outcome.classes_]
model_df[search_rf_play_class_names] = pd.DataFrame(search_rf_play_outcome.predict_proba(model_df[mask_model_predict][input_names]), index=model_df[mask_model_predict].index)
model_df[search_rf_play_class_names] = model_df[search_rf_play_class_names].fillna(0)
model_df[search_rf_drive_class_names] = pd.DataFrame(search_rf_drive_outcome.predict_proba(model_df[mask_model_predict][input_names]), index=model_df[mask_model_predict].index)
model_df[search_rf_drive_class_names] = model_df[search_rf_drive_class_names].fillna(0)

search_rf_play_class_names_home = [x + "_home" for x in search_rf_play_class_names]
search_rf_play_class_names_away = [x + "_away" for x in search_rf_play_class_names]
search_rf_drive_class_names_home = [x + "_home" for x in search_rf_drive_class_names]
search_rf_drive_class_names_away = [x + "_away" for x in search_rf_drive_class_names]
model_df[search_rf_play_class_names_home] = model_df[search_rf_play_class_names].where(model_df.home_team_has_ball==1, 0)
model_df[search_rf_play_class_names_away] = model_df[search_rf_play_class_names].where(model_df.home_team_has_ball==0, 0)
model_df[search_rf_drive_class_names_home] = model_df[search_rf_drive_class_names].where(model_df.home_team_has_ball==1, 0)
model_df[search_rf_drive_class_names_away] = model_df[search_rf_drive_class_names].where(model_df.home_team_has_ball==0, 0)
display_html(model_df[mask_model][search_rf_play_class_names_home].head(15))
display_html(model_df[mask_model][search_rf_drive_class_names_home].head(15))

,search_rf_play_field_goal_made_home,search_rf_play_field_goal_missed_home,search_rf_play_first_down_home,search_rf_play_none_home,search_rf_play_offensive_touchdown_home,search_rf_play_turnover_home
2,0.000000e+00,0.000000,0.177257,0.797776,0.006842,0.018125
3,2.762011e-07,0.000005,0.269195,0.703363,0.006831,0.020607
4,0.000000e+00,0.000003,0.344761,0.608076,0.009020,0.038140
5,1.224140e-06,0.000000,0.184944,0.789976,0.009224,0.015855
6,2.762011e-07,0.000006,0.298935,0.670069,0.009723,0.021267
7,1.224140e-06,0.000000,0.189624,0.782763,0.011211,0.016400
8,1.094359e-05,0.000000,0.104436,0.819032,0.062322,0.014200
9,3.027111e-04,0.000062,0.274966,0.643217,0.059572,0.021879
10,2.973331e-04,0.000062,0.309201,0.588757,0.068857,0.032825
11,2.617972e-07,0.000000,0.018438,0.777139,0.194211,0.010212


,search_rf_drive_clock_home,search_rf_drive_field_goal_made_home,search_rf_drive_field_goal_missed_home,search_rf_drive_punt_home,search_rf_drive_safety_home,search_rf_drive_touch_down_home,search_rf_drive_turnover_home,search_rf_drive_turnover_on_downs_home
2,0.001627,0.116460,0.022646,0.543364,0.003435,0.175361,0.120722,0.016386
3,0.001256,0.110399,0.020726,0.578062,0.002997,0.159085,0.111848,0.015627
4,0.000609,0.071220,0.012759,0.711120,0.003036,0.101360,0.088590,0.011305
5,0.001456,0.137521,0.025099,0.495564,0.001075,0.204051,0.115819,0.019414
6,0.001254,0.131096,0.022911,0.532615,0.001048,0.184651,0.108144,0.018281
7,0.001110,0.316343,0.058961,0.124513,0.000043,0.369347,0.095493,0.034190
8,0.000937,0.371114,0.046645,0.034786,0.000040,0.449162,0.072568,0.024747
9,0.000916,0.388575,0.043961,0.035553,0.000034,0.437526,0.067151,0.026285
10,0.000479,0.489969,0.065972,0.045532,0.000047,0.313843,0.053709,0.030450
11,0.000539,0.277402,0.019524,0.018600,0.000025,0.611744,0.047145,0.025020


### Score Difference Model
* Using the same inputs and adding the outputs of the previous model, we will predict the score differential probablities
* Since this model is an MLP model, we will normalize the inputs. (all features will be made so that the range is from 0 to 1)
* In this model, continuation, null values, end of quarters, and overtime is removed.

In [136]:
search_mlp_score_diff_clipped_rf_drive_preds = pickle.load(open(os.path.join(root_dir, "models/search_mlp_score_diff_clipped_rf_drive_preds.p"), 'rb'))
model_df["end_of_regulation_score_diff_change_clipped"] = np.clip(model_df["end_of_regulation_score_diff_change"], -35, 35)


input_names_score_pred = [item for item in input_names if item not in ["punt", "field_goal_attempt"]] + ["kick_off", "point_after_kick", "two_point_attempt"] + ["search_rf_play_first_down_home", "search_rf_play_first_down_away"] + search_rf_drive_class_names_home[1:] + search_rf_drive_class_names_away[1:]
output_name = "end_of_regulation_score_diff_change_clipped"

mask_model_score_diff = (
    (model_df.continuation==0)&
    (model_df[input_names_score_pred+[output_name]].notna().all(axis=1))&
    ~(model_df.event_id.isin([12,57,58,13]))&
    (model_df["overtime"]==0)
)


normalized_score_pred_df = normalize_df(model_df[mask_model_score_diff][input_names_score_pred], model_df[mask_model_score_diff & (model_df.season<2020)][input_names_score_pred])
mlp_search_score_diff_clipped_rf_drive_preds_preds = pd.DataFrame(search_mlp_score_diff_clipped_rf_drive_preds.predict_proba(normalized_score_pred_df.values), index=model_df[mask_model_score_diff].index)
score_diff_clipped_rf_drive_preds_matrix = pd.DataFrame(np.zeros(mlp_search_score_diff_clipped_rf_drive_preds_preds.shape), index=mlp_search_score_diff_clipped_rf_drive_preds_preds.index)
score_diff_change_list_clipped = list(model_df.end_of_regulation_score_diff_change_clipped.drop_duplicates().sort_values())

for column in score_diff_clipped_rf_drive_preds_matrix.columns:
    score_diff_clipped_rf_drive_preds_matrix[column] = score_diff_change_list_clipped[column] + model_df["current_score_diff"]

model_df["xhome_win_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T>0], axis=0)
model_df["xovertime_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T==0], axis=0)
model_df["xaway_win_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T<0], axis=0)
model_df["xend_of_regulation_score_diff_mlp_search_clipped_rf_drive_preds"] = np.sum(score_diff_clipped_rf_drive_preds_matrix * mlp_search_score_diff_clipped_rf_drive_preds_preds, axis=1)

display_html(model_df[["xhome_win_mlp_search_clipped_rf_drive_preds", "xovertime_mlp_search_clipped_rf_drive_preds", "xaway_win_mlp_search_clipped_rf_drive_preds"]].dropna())
display_html(pd.DataFrame(mlp_search_score_diff_clipped_rf_drive_preds_preds.values, columns=score_diff_change_list_clipped))

,xhome_win_mlp_search_clipped_rf_drive_preds,xovertime_mlp_search_clipped_rf_drive_preds,xaway_win_mlp_search_clipped_rf_drive_preds
0,0.640969,0.058230,0.300802
2,0.615716,0.062351,0.321933
3,0.617222,0.062212,0.320565
4,0.608531,0.062546,0.328923
5,0.638916,0.061036,0.300049
...,...,...,...
816791,0.093305,0.120986,0.785709
816792,0.073841,0.094863,0.831295
816793,0.059803,0.052311,0.887887
816795,0.026669,0.019269,0.954062


,-35,-34,-33,-32,-31,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,4.999944e-04,7.324320e-04,2.787122e-04,2.519889e-04,1.273351e-03,5.792011e-04,2.303283e-04,1.503494e-03,1.055686e-03,5.921833e-04,1.136500e-03,4.302760e-03,2.430983e-03,1.530411e-03,0.004727,0.004724,2.340620e-03,0.005092,0.008595,0.005143,0.004218,0.010960,0.008953,0.005095,0.009091,0.014944,0.009493,0.012321,0.032555,0.021267,0.016039,0.029857,0.035436,0.017944,0.025609,0.058230,0.030616,0.024285,0.053730,0.035476,0.017514,0.026877,0.056871,0.021800,0.016718,0.042195,0.022797,0.011746,0.020181,0.036283,0.010918,0.014768,0.024821,0.016540,6.945037e-03,0.019009,0.023415,7.260666e-03,9.472296e-03,1.684311e-02,6.520225e-03,5.473751e-03,9.475109e-03,1.205546e-02,2.496027e-03,4.891446e-03,8.338373e-03,2.772812e-03,2.275318e-03,4.381212e-03,1.520763e-02
1,9.949549e-04,1.540929e-03,4.487812e-04,4.603821e-04,1.898866e-03,1.043979e-03,5.791226e-04,2.327790e-03,1.511514e-03,8.464800e-04,1.601760e-03,5.358951e-03,3.540980e-03,2.086576e-03,0.005926,0.005675,2.752907e-03,0.005938,0.009305,0.006065,0.004949,0.012822,0.009465,0.005691,0.009168,0.014769,0.009429,0.013321,0.035650,0.020931,0.016339,0.031608,0.035634,0.017191,0.025063,0.062351,0.026547,0.023684,0.057565,0.030756,0.018374,0.026348,0.060670,0.021442,0.015447,0.039433,0.018254,0.010763,0.018756,0.032606,0.010338,0.015589,0.021178,0.014632,6.930026e-03,0.017695,0.019361,6.689194e-03,9.360275e-03,1.377637e-02,5.568314e-03,6.227034e-03,9.334123e-03,1.227247e-02,2.863246e-03,6.039041e-03,8.882946e-03,2.850993e-03,3.150255e-03,4.642164e-03,1.768987e-02
2,9.722643e-04,1.481627e-03,4.423749e-04,4.497983e-04,1.849925e-03,1.011195e-03,5.517870e-04,2.294300e-03,1.474545e-03,8.265493e-04,1.601436e-03,5.274599e-03,3.447589e-03,2.056299e-03,0.005919,0.005573,2.731191e-03,0.005913,0.009280,0.006014,0.004931,0.012856,0.009390,0.005676,0.009272,0.014851,0.009371,0.013300,0.035324,0.020662,0.016249,0.031690,0.035565,0.017060,0.025202,0.062212,0.026420,0.023509,0.057656,0.031003,0.018277,0.026623,0.061149,0.021312,0.015545,0.039881,0.018547,0.010796,0.019031,0.032950,0.010289,0.015485,0.021585,0.014675,6.859008e-03,0.017818,0.019560,6.634652e-03,9.314374e-03,1.385601e-02,5.531556e-03,6.172661e-03,9.303336e-03,1.217950e-02,2.827003e-03,5.917162e-03,8.848348e-03,2.789790e-03,3.066241e-03,4.585283e-03,1.722567e-02
3,1.021217e-03,1.522522e-03,4.653744e-04,4.716632e-04,1.919474e-03,1.060572e-03,5.795609e-04,2.387596e-03,1.557371e-03,8.768821e-04,1.680651e-03,5.553969e-03,3.604037e-03,2.140529e-03,0.006180,0.005851,2.835449e-03,0.006131,0.009738,0.006270,0.005047,0.013371,0.009807,0.005833,0.009521,0.015452,0.009654,0.013550,0.036273,0.021273,0.016394,0.031871,0.036388,0.017366,0.025276,0.062546,0.026879,0.023217,0.056928,0.031342,0.018443,0.026313,0.060592,0.021342,0.015245,0.039457,0.018512,0.010670,0.018718,0.032186,0.010262,0.015107,0.021059,0.014562,6.832736e-03,0.017283,0.019094,6.591002e-03,8.922112e-03,1.344913e-02,5.375313e-03,5.951705e-03,8.979078e-03,1.174066e-02,2.766062e-03,5.612761e-03,8.491477e-03,2.645178e-03,2.987988e-03,4.479575e-03,1.649487e-02
4,7.233916e-04,1.142692e-03,3.356056e-04,3.473600e-04,1.560505e-03,7.864509e-04,4.032161e-04,1.880943e-03,1.232127e-03,6.707943e-04,1.294771e-03,4.495215e-03,2.904473e-03,1.766497e-03,0.005059,0.004866,2.400589e-03,0.005340,0.008266,0.005248,0.004442,0.011374,0.008508,0.005182,0.008586,0.013744,0.008877,0.012474,0.033572,0.019981,0.015781,0.030893,0.034215,0.016749,0.024945,0.061036,0.026739,0.024736,0.058308,0.031088,0.017960,0.027278,0.060245,0.021799,0.016592,0.040992,0.019203,0.011470,0.019867,0.034933,0.010607,0.016321,0.022936,0.015182,7.118164e-03,0.019016,0.020929,6.912398e-03,1.019560e-02,1.520786e-02,6.112592e-03,6.323175e-03,1.006470e-02,1.290870e-02,2.890825e-03,6.298571e-03,9.268728e-03,3.105516e-03,3.08

In [144]:
model_df[mask_model_score_diff &(model_df.ytg==-1)]["event_name"].value_counts()

Kick Off                 37300
Point after Touchdown    17803
Onside KO                  894
2 Pt Pass Completion       487
2 Pt Incomplete Pass       483
2 Pt Run                   370
Offensive Penalty          179
Blocked PAT Try            131
Defensive Penalty          109
Fumble                      46
2 Pt Sack                   20
2 Pt Interception           17
Pass Completion              2
Incomplete Pass              2
Run                          2
Lateral                      1
Sack                         1
Field Goal Attempt           1
Name: event_name, dtype: int64